In [1]:
import chess.pgn
import chess
import chess.svg
import requests


In [2]:
# Setzen Sie Ihren Lichess Benutzernamen und den Ausgabedateipfad
username = "SchachlordZlatan"
file = "alle_spiele.pgn"

In [3]:
def download_all_games(username, output_file):
    url = f"https://lichess.org/api/games/user/{username}"
    response = requests.get(url)

    if response.status_code == 200:
        with open(output_file, 'wb') as f:
            f.write(response.content)
        print(f"Alle Spiele von {username} wurden erfolgreich heruntergeladen und in {output_file} gespeichert.")
    else:
        print(f"Fehler beim Herunterladen der Spiele von {username}. Statuscode: {response.status_code}")

# download_all_games(username, file)


In [13]:
def filter_won_games(pgn_file, player_name):
    with open(pgn_file) as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            headers = game.headers
            result = headers["Result"]
            white = headers["White"]
            black = headers["Black"]

            if result.startswith("1-0") and white == player_name:
                yield game

            elif result.startswith("0-1") and black == player_name:
                yield game


# won_games = filter_won_games(file, username)

# for game in won_games:
#     print(game.headers["Event"])

In [20]:
def filter_games_with_knight_check(games):
    filtered_games = []

    for game in games:
        board = game.board()
        has_knight_check = False

        for move in game.mainline_moves():
            board.push(move)

            # Überprüfen, ob das Pferd ein Schach gibt
            if board.is_check():
                king_square = board.king(board.turn)
                attackers = board.attackers(not board.turn, king_square)
                for attacker_square in attackers:
                    if board.piece_at(attacker_square).piece_type == chess.PieceType.KNIGHT:
                        has_knight_check = True
                        break

            if has_knight_check:
                filtered_games.append(game)
                break

    return filtered_games

# Beispielaufruf
games = list(filter_won_games(file, username))
filtered_games = filter_games_with_knight_check(games)

for game in filtered_games:
    print(game.headers["Event"])



728
